<a href="https://colab.research.google.com/github/Noors-lab/pytorch_stuffs/blob/main/intuition_code_for_MultiHead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# the libraries


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Notation and input

 batch_size = B
 sequence_length = T
 Model (embedding) dimensions = d_model
 number of heads = h
 per head dimensions = d_model/h
 input tensor
 X_shape = [B,T,d_model]

Trainable linear projections (these are nn.Linear layers in code):

W_q, W_k, W_v: each maps d_model > d_model

W_o: final output projection d_model > d_model


# Compute Q,K,V

In [ ]:
Q = X @ W_q   # shape (B, T, d_model)
K = X @ W_k   # shape (B, T, d_model)
V = X @ W_v   # shape (B, T, d_model)

# then split into heads by reshaping and transposing

Q = Q.view(B, T, h, d_head).transpose(1,2)  # -> (B, h, T, d_head)
K = K.view(B, T, h, d_head).transpose(1,2)  # -> (B, h, T, d_head)
V = V.view(B, T, h, d_head).transpose(1,2)  # -> (B, h, T, d_head)

#We want to compute attention per head;
#the h axis groups heads so operations can be batched.

# scaled dot products

In [ ]:
scores = Q @ K.transpose(-2, -1)   # shape -> (B, h, T, T)

#Each scores[b, head, i, j] is the raw dot product between token i's query and token j's key in that head.

#then we do scaling with sqrt(d_head) to avoid large dot products that pushe softmax to extreme value.

scores = scores / sqrt(d_head)


# softmax and weights

In [ ]:
weights = softmax(scores, dim=-1)   # shape (B, h, T, T)

#Each row weights[b, head, i, :] sums to 1
#and tells how token i distributes attention over all tokens.

#   WEIGHTED SUM VALUE

head_out = weights @ V   # shape -> (B, h, T, d_head)

#This produces the attended representation per head.

# recombining the heads

In [ ]:
head_out = head_out.transpose(1,2).reshape(B, T, d_model)  # -> (B, T, d_model)

# project with W_o

output = head_out @ W_o   # (B, T, d_model)

# this is the final multihead output layer